In [ ]:
from sqlalchemy import create_engine, MetaData
import pandas as pd

user = ''
passw = ''
host = ''
port = ''

database = 'schemas'
tabela = 'ecommerce'
mydb = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database, echo=False)
# ECOMMERCE = pd.read_sql('SELECT * FROM `' + tabela + '`', con=mydb)

METADATA_OBJ = MetaData()
METADATA_OBJ.reflect(mydb)
METADATA_OBJ.tables.keys()

mydb.dispose()

In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

modelo = 'llama-3.1-70b-versatile'
modelo_hf_emb = 'BAAI/bge-m3'

groq_key = ''
Settings.llm = Groq(model=modelo, api_key=groq_key)
Settings.embed_model = HuggingFaceEmbedding(model_name=modelo_hf_emb)

#### PARA ACESSAR O BANCO E ENTENDER O BANCO PRECISAMOS ESTRUTURAR O BANCO COMO UM OBJETO LLAMAINDEX (SQLDATABASE)

In [ ]:
# PARA CRIAR O OBJETO PRECISAMOS DO SQLDATABASE E SQLTABLENODEMAPPING
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping

DATABASE = SQLDatabase(mydb)
TABLE_NODE_MAP = SQLTableNodeMapping(DATABASE)

#### ESQUEMATIZANDO AS TABELAS PARA O FORMATO DE OBJETO ESPECIFICO PARA O LLAMAINDEX

In [ ]:
from llama_index.core.objects import SQLTableSchema

TABLE_SCHEMA_OBJ = []

for tables in METADATA_OBJ.tables.keys():
    TABLE_SCHEMA_OBJ.apped(SQLTableSchema(table_name=tables))

#### OBTENDO INFORMAÇÕES (INDICES VETORIAIS) DO DATABASE ATRAVES DO EMBEDDING DO HUGGING FACE

In [ ]:
from llama_index.core.objects import ObjectIndex
from llama_index.core import VectorStoreIndex

OBJ_INDEX = ObjectIndex.from_objects(TABLE_SCHEMA_OBJ, TABLE_NODE_MAP, VectorStoreIndex)

#### O RETRIEVER É UM OBJETO QUE NOS AJUDA A TRANSFORMAR A SEMÂNTICA DA PERGUNTA EM UMA CONSULTA NO BANCO

In [ ]:
# A PARTIR DA PERGUNTA, ELE RELACIONA A PALAVRA MAIS ADEQUADA AO NOME DA TABELA PARA FAZER A CONSULTA (SOMENTE O MAIS RELEVANTE similarity_top_k=1)
OBJ_RETRIEVER = OBJ_INDEX.as_retriever(similarity_top_k=1)

In [ ]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

QUERY_ENGINE = SQLTableRetrieverQueryEngine()